Attention Mask : 실제 토큰 1/0 패딩  
Token Type IDS(Segment IDs) -> 한 문장인지 두 문장인지 BERT에게 알려주는 문장구분번호 -> 문장 구분을 위한 추가 레이어  
CLS Token Pooling : [CLS] + token + [SEP]  


<span style="color: Gold"> 1. BERT TOKENIZER </span>: 단어를 의미있는 조각(subword)로 나눈다.  
unblievable "un" "believ" "able"   

In [ ]:
from transformers import BertTokenizer

# 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # uncase 는 소문자로 변환/ case는 대소문자 변환
sentences = [
    'Hello, world!',
    'unbelievable performance!',
    'COVID-19 pandemic'
]
for sentence in sentences:
  # 토큰화
  tokens = tokenizer.tokenize(sentence)
  print(f'원문 : {sentence}')  
  print(f'원문 : {tokens}')  

  # ID 변환
  ids = tokenizer.convert_tokens_to_ids(tokens)
  print(f'원문 : {ids}')

  # 역변환
  decoded_str = tokenizer. decode(ids)
  print(f'역변환 : {decoded_str}\n')

원문 : Hello, world!
원문 : ['hello', ',', 'world', '!']
원문 : [7592, 1010, 2088, 999]
역변환 : hello, world!

원문 : unbelievable performance!
원문 : ['unbelievable', 'performance', '!']
원문 : [23653, 2836, 999]
역변환 : unbelievable performance!

원문 : COVID-19 pandemic
원문 : ['co', '##vid', '-', '19', 'pan', '##de', '##mic']
원문 : [2522, 17258, 1011, 2539, 6090, 3207, 7712]
역변환 : covid - 19 pandemic



-> 원문은 토큰화(숫자로변환)하고 다시 읽을 수 있는 문자로 바꿈(역변환)  
대문자도 모두 소문자로 변환

---

<span style="color: Gold"> 2. Attention Mask
- 패딩이 attention 계산에 섞여서 모델이 혼란을 일으켜 불필요한 연산이 증가하고 노이즈가 증가한다  
-> `토큰이 실제 단어면 1, 패딩이면 0으로 표시하여 패딩을 무시하라고 알려주는 것`


In [3]:
# 어텐션마스크
from transformers import BertTokenizer
# 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
sentences = [
    'short sentence',
    'This is a much longer sentence with more words'
]
# 여러문장을 한꺼번에 토크나이징하고 가장 긴 문장길이에 맞춰 자동 패딩 수행
encoded = tokenizer(
    sentences,
    padding = True,
    return_tensors = 'pt' # 토크나이저가 출력하는 결과를 파이토치 텐서(torch.Tensor) 형태로 만들어라 -> 모델(BERT, GPT 등)은 텐서(tensor) 형식만 받아서 연산
)
encoded

{'input_ids': tensor([[ 101, 2460, 6251,  102,    0,    0,    0,    0,    0,    0,    0],
        [ 101, 2023, 2003, 1037, 2172, 2936, 6251, 2007, 2062, 2616,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

---

<span style="color: Gold"> 3. token_type_ids
- 한 문장인지 두 문장인지 BERT에게 알려주는 문장구분번호 -> 문장 구분을 위한 추가 레이어  

In [8]:
from transformers import BertTokenizer
# 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
sentence_A = 'The weather is nice'
sentence_B = "Let's fo for a work"
# 두 문장을 하나의 입력으로 인코딩
encoded = tokenizer(
    sentence_A,
    sentence_B,
    padding = True,
    return_tensors = 'pt'
)
encoded
# 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]]) A를 0, B를 1로 변환하여 다른 문장임을 표시
tokens = tokenizer.convert_ids_to_tokens(encoded['input_ids'][0])
# input_ids': tensor([[ 101, 1996, 4633, 2003, 3835,  102, 2292, 1005, 1055, 1042, 2080, 2005,1037, 2147,  102]])
# 4글자말고 3글자가 특수토큰이라는 것을 알 수 있음. 101은 시작, 102는 문장의 끝이자 다른 문장의 시작
print(tokens)
for token, token_id, type_id in zip(tokens,encoded['input_ids'][0],encoded['token_type_ids'][0]):
  segment = '문장 A' if type_id == 0 else '문장 B'
  if token == '[SEP]': 
    segment = '구분자'
  elif token == '[CLS]':
    segment = '시작'
  print(f'{token:20s}: {token_id.item():6d}({type_id.item():6d})({segment})')

['[CLS]', 'the', 'weather', 'is', 'nice', '[SEP]', 'let', "'", 's', 'f', '##o', 'for', 'a', 'work', '[SEP]']
[CLS]               :    101(     0)(시작)
the                 :   1996(     0)(문장 A)
weather             :   4633(     0)(문장 A)
is                  :   2003(     0)(문장 A)
nice                :   3835(     0)(문장 A)
[SEP]               :    102(     0)(구분자)
let                 :   2292(     1)(문장 B)
'                   :   1005(     1)(문장 B)
s                   :   1055(     1)(문장 B)
f                   :   1042(     1)(문장 B)
##o                 :   2080(     1)(문장 B)
for                 :   2005(     1)(문장 B)
a                   :   1037(     1)(문장 B)
work                :   2147(     1)(문장 B)
[SEP]               :    102(     1)(구분자)


---

<span style="color: Gold"> [CLS]Token Pooling : 
- BERT 첫번째 토큰 [CLS] 문서 전체의 요약 => 분류 작업을 할때  
이 토큰의 출력만 가져와서 분류기 (classifier)에 연결

In [ ]:
from transformers import BertTokenizer,BertModel
import torch
# 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
sentence = "BERT is amazing for NLP tasks!"
# 인코딩
inputs = tokenizer(sentence,return_tensors = 'pt') # 'return_tensorse' -> 'return_tensors' 수정
# BERT 통과
with torch.no_grad():
  outputs = model(**inputs)
# 출력 형태 확인
last_hidden_state = outputs.last_hidden_state # 문장 가장 마지막에 쌓인 state
print(f'입력문장 : {sentence}')
print(f'last_hidden_state 형태 : {last_hidden_state.shape}')
print(f'batch_size = 1 sequence_length : {last_hidden_state.shape[1]} hidden_size = {last_hidden_state.shape[2]}')
# [CLS] 토큰 추출
cls_embedding = last_hidden_state[:,0,:]    # last_hidden_state : [batch_size, seq_len, hidden_size]/ 
                                            # batch_size = 1 (문장 1개), seq_len = 문장 토큰 수, hidden_size = 768 (BERT-base)
                                            # [:,0,:] 의미:/   : → batch 전체 선택/   0 → 첫 번째 토큰([CLS])/  : → hidden_size 전체 선택
print(f'cls_embedding 형태 : {cls_embedding.shape}')
# 분류기 (2-class)
classifier = torch.nn.Linear(768,2) # Linear 선형변환 / 입력: [CLS] 벡터 768차원 / 출력: 2차원 logits (클래스 2개) 긍정,부정
logits = classifier(cls_embedding)  # (1,2) (batch, class개수)
probs = torch.softmax(logits, dim = -1) # softmax → logits → 확률로 변환 / dim=-1 → 마지막 차원 기준으로 계산 (클래스 차원)
print(f'logits = {logits}')
print(f'probs = {probs}')
print(f'predicted class = {torch.argmax(probs).item()}')    # argmax : 가장 큰 값의 인덱스(위치)를 반환하는 함수/ -> 즉 확률이 가장 높을 것을 가져오는 것

입력문장 : BERT is amazing for NLP tasks!
last_hidden_state 형태 : torch.Size([1, 10, 768])
batch_size = 1 sequence_length : 10 hidden_size = 768
cls_embedding 형태 : torch.Size([1, 768])
logits = tensor([[ 0.0050, -0.0674]], grad_fn=<AddmmBackward0>)
probs = tensor([[0.5181, 0.4819]], grad_fn=<SoftmaxBackward0>)
predicted class = 0


---

<span style="color: Gold"> 미세 조정 학습 Fine_tuning

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
texts = [
    "This movie is fantastic!",
    "Terrible film, waste of time.",
    "Amazing plot and great acting.",
    "Boring and predictable."
]
labels = [1, 0, 1, 0]  # 1=positive, 0=negative

# 토크나이져 모델
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# 모델
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)
# 데이터셋
class SimpleDataset(Dataset):
  def __init__(self, texts, labels):
    self.encodings = tokenizer(texts, truncation=True, padding=True, return_tensors='pt')
    self.labels = labels
  def __getitem__(self, idx):
    item = {key: val[idx] for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item
  def __len__(self):
    return len(self.labels)
dataset = SimpleDataset(texts,labels)
loader = DataLoader(dataset, batch_size=2)
# 학습설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)
# 미세조정(학습)
model.train()
for epoch in range(20):
  total_loss = 0
  for batch in loader:
    optimizer.zero_grad()
    inputs = { k:v.to(device) for k,v in batch.items() if k != 'labels'}
    labels = batch['labels'].to(device)
    outputs = model(**inputs, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  print(f'epoch : {epoch+1}, loss : {total_loss/len(loader)}')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


epoch : 1, loss : 0.7223823070526123
epoch : 2, loss : 0.6858187019824982
epoch : 3, loss : 0.6633428931236267
epoch : 4, loss : 0.5623259544372559
epoch : 5, loss : 0.5752913355827332
epoch : 6, loss : 0.5011169910430908
epoch : 7, loss : 0.47055280208587646
epoch : 8, loss : 0.47721824049949646
epoch : 9, loss : 0.3633720725774765
epoch : 10, loss : 0.4408431947231293
epoch : 11, loss : 0.4125638008117676
epoch : 12, loss : 0.3528033047914505
epoch : 13, loss : 0.40727663040161133
epoch : 14, loss : 0.2697630375623703
epoch : 15, loss : 0.28098955750465393
epoch : 16, loss : 0.254472091794014
epoch : 17, loss : 0.2099958211183548
epoch : 18, loss : 0.23029977083206177
epoch : 19, loss : 0.18939511477947235
epoch : 20, loss : 0.20245113968849182


---

추론

In [12]:
model.eval() # 평가모드
sample_sentences= [
"I am really disappointed with the result.",
"The service was terrible and not worth the money.",
"I don't like this product at all."
]
# 토큰화
inputs = tokenizer(
    sample_sentences,
    truncation = True,
    padding = True,
    return_tensors = 'pt'
)

# gpu, cpu 설정
inputs = {k: v.to(device) for k,v in inputs.items()}
# 추론
with torch.no_grad():
  outputs = model(**inputs)
  logits = outputs.logits
  probs = torch.softmax(logits, dim=-1)
  pred = torch.argmax(probs, dim=-1).detach().numpy()
probs, pred

(tensor([[0.3854, 0.6146],
         [0.4785, 0.5215],
         [0.4353, 0.5647]]),
 array([1, 1, 1]))